In [45]:
# Imports
import pandas as pd
from datetime import datetime, timedelta
import time, requests
import plotly.express as px
import matplotlib.pyplot as plt

In [46]:
def epoch_to_datetime(epoch_date: int):
    if len(str(epoch_date)) > 10:
        epoch_date = str(epoch_date)[:-3]
    timeArray = time.localtime(int(epoch_date))
    return time.strftime("%Y-%m-%d %H:%M:%S", timeArray)

def get_open_interest(symbol: str, interval: str):
    url = "{}/api/v1/contract_his_open_interest?symbol={}&contract_type=quarter&period={}&amount_type=1".format("https://api.hbdm.com", symbol, interval)
    df_oi = pd.DataFrame(requests.get(url).json()["data"]["tick"])
    df_oi['ts'] = df_oi['ts'].astype(str)
    for index, row in df_oi.iterrows():
        df_oi.at[index,'ts'] = str(epoch_to_datetime(int(row['ts'])))

    df_oi.columns  = ['Volume', 'Amount', 'Date']
    return df_oi
     

def get_top_account_ratios(symbol: str, interval: str):
    url = "{}/api/v1/contract_elite_position_ratio?symbol={}&period={}".format("https://api.hbdm.com", symbol, interval)
    df_account_ratios = pd.DataFrame(requests.get(url).json()["data"]["list"])
    df_account_ratios['ts'] = df_account_ratios['ts'].astype(str)
    for index, row in df_account_ratios.iterrows():
        df_account_ratios.at[index,'ts'] = str(epoch_to_datetime(int(row['ts'])))

    df_account_ratios.columns  = ['Buy Ratio', 'Sell Ratio', 'Date']

    return df_account_ratios

In [47]:
symbol="BTC"
interval = "1day"

In [48]:
df_oi = get_open_interest(symbol, interval)
df_account_ratios = get_top_account_ratios(symbol, interval)

In [49]:
# Open Interest
fig = px.line(df_oi , x="Date", y="Volume", title="{} Huobi Open Interest".format(symbol), template = "plotly_dark")
fig.show()

In [51]:
# Top Accounts side ratios
fig = px.line(df_account_ratios , x="Date", y=["Buy Ratio","Sell Ratio"], title="{} Huobi Top Account Side ratios".format(symbol), template = "plotly_dark")
fig.update_layout(legend_title="Side")
fig.show()